# Motion planning

In this tutorial we will implement a RRT algorithm for a reduced model of the ur5 robot with only 2 degrees of freedom. This will allow us to visualise the 2D configuration space of the environment, as we saw in tutorial 0. We will then use the same code to plan a path for the complete robot. In tutorial 0, we were using local methods to find a path from one configuration to the other, with a significant failure rate. RRT, as a global planning method, is probabilistically complete, ie, if a solution exists, it is guaranteed to find it, eventually (this can take a while...)

**Important: to make sure your repository is easily updated from git shall it be needed, create a copy of this notebook before working on it.**

## The RRT algorithm
the pseudo-code for the RRT algorithm is rather simple (source: [wikipedia](https://en.wikipedia.org/wiki/Rapidly_exploring_random_tree)). RRT comes with a large variety of implementations and adaptations, depending on the targeted application.


<img src="tp3/rrt_pseudo.png" style="width: 700px;">

Given an initial configuration $q_{init}$,  a **valid** configuration $q_{rand}$ is generated randomly. What "valid" means is problem dependent. In our case we will consider a **valid** configuration to be one that is collision free and respects the joint limits of the robot.

Then, the **nearest** vertex $q_{near}$ from $q_{rand}$ in the graph $G$ is selected. Here again, the notion of "near" needs to be defined (especially when we consider free flyer joints). In our case we will simply consider the euclidian distance between the configurations.

NEW_CONF tries to connect $q_{rand}$ and $q_{near}$ with a local method (usually cheap to compute), in the hope of obtaining a **valid** path between $q_{rand}$ and $q_{near}$. Again, what a valid path can be chosen. In this tutorial, a valid path will be composed of **valid configurations**. If the distance between $q_{rand}$ and $q_{near}$ is greater than a user-defined variable $\Delta q$ and / or if the complete path is not valid, then the furthest configuration $q_{new}$ such that the path between $q_{rand}$ and $q_{new}$ is valid will be kept and added to the graph.

How is computed the path then? Again, more or less complex methods can be used. In this tutorial a path between two configurations will simply consist in their linear interpolation (thus, assuming no free-flyer or spherical joint on the robot!)

In this tutorial we will use a variation of this graph generation, called single-query RRT. We will try to connect each new configuration to a desired goal position and stop the generation if we successfully do so.


<img src="tp3/rrtquary_pseudo.png">

## Robot model
Although our code will be independent from the robot, we will load one now for visualisation purposes

In [15]:
import magic_donotload

In [16]:
import pinocchio as pin #the pinocchio library
from utils.meshcat_viewer_wrapper import MeshcatVisualizer # the meshcat visualiser
from utils.load_ur5_with_obstacles import load_ur5_with_obstacles,Target # helper function to load scene
from scipy.optimize import fmin_bfgs,fmin_slsqp # some optimisation routines from scipy
import time # the remaining libraries are python native
import numpy as np 
from numpy.linalg import norm
import matplotlib.pylab as plt 

robot = load_ur5_with_obstacles(reduced=True)

viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


At the moment, our robot only has 2 degrees of freedom (we have blocked the others). This will help us with the visualisation in a first instance.

In [17]:
print(robot.q0)

[0. 0.]


## Implementation of the Graph
We will use a very simple graph structure that will allow us to retrieve the path from the initial state to the goal. Because we are in a single query case, the graph will be directed. Whether or not the graph is directed usually depends, again, on the problem. For instance, a path connecting two configurations for a car is not necessarily invertible.

Our graph will be simply implemented as a list of Nodes.

A Node is a duplet (parent, q), where parent is an integer pointing to its parent node in the graph and q is the configuration.

For instance the graph:

3 <------ 0 --> 1 --> 2
        
will be implemented as 

G = [(None,q0),(0,q1),(1,q2),(0,q3)]

Where the qis are relevant configurations

## Implementation of the RRT methods
Let's start with the RAND_CONF method. We need to sample a random configuration. Pinocchio automatically does it for you, ensuring that the configuration respects the joint limits: 

### RAND_CONF

In [19]:
viz.display(pin.randomConfiguration(robot.model))


We additionally need to check that a configuration is free of collisions. The following function does it for you:

In [20]:
# %load tp0/generated/simple_path_planning_coll
def coll(q):
     '''Return true if q in collision, false otherwise.'''
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     return pin.computeCollisions(robot.collision_model,robot.collision_data,False)


We can now easily write *RAND_CONF*:

In [21]:
def RAND_CONF():
    '''
    Return a random configuration, not is collision
    '''
    while True:
        q = pin.randomConfiguration(robot.model)  # sample between -3.2 and +3.2.
        if not coll(q): return q
        
q = RAND_CONF()
viz.display(q)

### NEAREST_VERTEX

For this tutorial, we will use a naive implementation of NEAREST_VERTEX. We will iterate through all the nodes in the graph and return the one who is the closest according to a distance function that you will write as well:

In [22]:
def distance(q1,q2):    
     '''Return the euclidian distance between two configurations'''
        
def NEAREST_VERTEX(G,q_rand):
     '''returns the index of the Node of G with the configuration closest to q_rand  '''
    

In [23]:
%do_not_load tp3/generated/rrt_nearest

In [37]:
# %load tp4/generated/rrt_nearest
def distance(q1,q2):    
    '''Return the euclidian distance between two configurations'''
    return np.linalg.norm(q2-q1)
        
def NEAREST_VERTEX(G,q_rand):
    '''returns the index of the Node of G with the configuration closest to q_rand  '''
    min_dist = 10e4
    idx=-1
    for (i,node) in enumerate(G):
        dist = distance(node[1],q_rand) 
        if dist < min_dist:
            min_dist = dist
            idx = i
    return idx


In our particular case, adding a vertex only happens when an edge is added as well, so we might as well just do it in one go:

In [73]:
def ADD_EDGE_AND_VERTEX(G,parent,q):
    G += [(parent,q)]

### VALID_EDGE and NEW_CONF
We have decided that a path between two nodes in the graph is computed as the linear interpolation between the two configurations. How to verify that the path is collision free?
The easiest way to do this is to discretise the path and verify at specific intervals that the configurations along it are collision free. What problems are associated with such a discretisation? 

Write the code for NEW_CONF and use it to write VALID_EDGE

In [74]:
def lerp(q0,q1,t):    
    return q0 * (1 - t) + q1 * t

def NEW_CONF(q_near,q_rand,discretisationsteps, delta_q = None):
    '''Return the closest configuration q_new such that the path q_near => q_new is the longest
    along the linear interpolation (q_near,q_rand) that is collision free and of length <  delta_q'''
    #TODO
    return q_near

def VALID_EDGE(q_near,q_goal,discretisationsteps):
    #TODO
    return False

In [75]:
def lerp(q0,q1,t):    
    return q0 * (1 - t) + q1 * t

def NEW_CONF(q_near,q_rand,discretisationsteps, delta_q = None):
    '''Return the closest configuration q_new such that the path q_near => q_new is the longest
    along the linear interpolation (q_near,q_rand) that is collision free and of length <  delta_q'''
    q_end = q_rand.copy()
    dist = distance(q_near, q_rand)
    if delta_q is not None and dist > delta_q:
        #compute the configuration that corresponds to a path of length delta_q
        q_end = lerp(q_near,q_rand,delta_q/dist)
        dist = delta_q
    dt = dist / discretisationsteps
    for i in range(1,discretisationsteps):
        q = lerp(q_near,q_end,dt*i)
        if coll(q):
            return lerp(q_near,q_end,dt*(i-1))
    return q_end


def VALID_EDGE(q_new,q_goal,discretisationsteps):
    return np.linalg.norm(q_goal -NEW_CONF(q_new, q_goal,discretisationsteps)) < 1

We now simply have to write the code equivalent to the pseudo code

In [81]:
discretisationsteps_newconf = 100 #TODO
discretisationsteps_validedge = 100 #TODO

slp = (1e-5)

def rrt(q_init, q_goal, k, delta_q):
    G = [(None,q_init)]
    for _ in range(k):
        q_rand = RAND_CONF()   
        q_near_index = NEAREST_VERTEX(G,q_rand)
        q_near = G[q_near_index][1]        
        q_new = NEW_CONF(q_near,q_rand,discretisationsteps_newconf, delta_q = None)    
        ADD_EDGE_AND_VERTEX(G,q_near_index,q_new)
        if VALID_EDGE(q_new,q_goal,discretisationsteps_validedge):
            print ("why")
            ADD_EDGE_AND_VERTEX(G,len(G)-1,q_goal)
            return G, True
    return G, False
            
q_init = RAND_CONF()
q_goal = RAND_CONF()
k = 1000
delta_q = 20.
       
G, res = rrt(q_init, q_goal, k, delta_q)

path = []

def plot(q0,q1,steps = 100):
    for i in range(steps):
        viz.display(lerp(q0,q1,float(i)/steps))
    viz.display(q1)
    

#if G is not None:
if res:
    node = G[-1]
    while node[0] is not None:
        path = [node[1]] + path
        print(node[0])
        node = G[node[0]]
        
    for q0, q1 in zip(path[:-1],path[1:]):
        plot(q0,q1,steps = 100)
        time.sleep(1e-3)
else:
    print("fail")


        

why
218
215
58
52
22
16
6
0


In [94]:
#if G is not None:
if res:        
    for q0, q1 in zip(path[:-1],path[1:]):
        plot(q0,q1,steps = 100)
        time.sleep(1e-3)

In [93]:
len(G)

220

In [45]:
q_init = G[3][1]
q_goal = G[0][1]
k = 100
delta_q = 20.
       
G, res = rrt(q_init, q_goal, k, delta_q)

path = []

def plot(q0,q1,steps = 100):
    for i in range(steps):
        viz.display(lerp(q0,q1,float(i)/steps))
    viz.display(q1)
    

#if G is not None:
if res:
    node = G[-1]
    while node[0] is not None:
        path = [node[1]] + path
        print(node[0])
        node = G[node[0]]
        
    for q0, q1 in zip(path[:-1],path[1:]):
        plot(q0,q1,steps = 100)
        time.sleep(1e-3)
else:
    print("fail")

VALID 0.2596656740436157
VALID 0.16477925032429397
VALID 0.16477925032429397
VALID 0.31087888421684806
VALID 0.2596656740436157
VALID 0.31284460975587264
VALID 0.43976380475328014
VALID 0.31087888421684806
VALID 0.31087888421684806
VALID 0.2596656740436157
VALID 0.2596656740436157
VALID 0.2596656740436157
VALID 1.4823792582008157
VALID 1.4823792582008157
VALID 1.4823792582008157
VALID 0.31087888421684806
VALID 0.2596656740436157
VALID 0.31087888421684806
VALID 0.2596656740436157
VALID 0.31087888421684806
VALID 0.31087888421684806
VALID 0.31087888421684806
VALID 0.2596656740436157
VALID 0.31087888421684806
VALID 1.4823792582008157
VALID 0.31087888421684806
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 1.4823792582008157
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 0.3379718904253293
VALID 0.2596656740436157
VALID 0.2596656740436157
VALID 1.4823792582008157
VALID 0.3379718904253293
VALID 1.180

In [46]:
viz.display(q_init)

In [47]:
viz.display(q_goal)